In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [2]:
%%time
path = '/glade/u/home/noteng/work/research/data/'
file = 'march13-march14.nc'
data = xr.open_dataset(path+file)
data = data.sel(time=slice('2020-03-13T04:00:00.000000000', '2020-03-14T05:00:00.000000000'))
data.close()

CPU times: user 28 ms, sys: 10.6 ms, total: 38.6 ms
Wall time: 443 ms


In [3]:
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,450:580,256:771] #Based on longitude and latitude of Andoya and Norwegian Sea
equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,250:650,450:850] #Based on longitude and latitude of Andoya and Norwegian Sea
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,330:580,660:780] #### hdm1 and hdm2
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor']
equivalent_reflectivity_factor

<xarray.DataArray 'equivalent_reflectivity_factor' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
    lon      (Yc, Xc) float32 ...
    lat      (Yc, Xc) float32 ...
Attributes:
    standard_name:  equivalent_reflectivity_factor
    long_name:      Radar Reflectivity
    units:          dBZ
    grid_mapping:   projection_lambert

In [4]:
# load Mask_ERF_files
Mask_ERF = iris.load('../saved-files/threshold-15/Mask_ERF_iris-15.nc')[0]
Mask_ERF

<iris 'Cube' of segmentation_mask / (1) (time: 301; -- : 400; -- : 400)>

In [5]:
track = pd.read_csv('../saved-files/threshold-15/track-reset-15.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,1,110.987364,214.505783,38,15,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.170987e+06,131505.783191,70.424947,13.466409,1,0 days 00:00:00
1,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00
2,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00
3,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00
4,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23064,300,9,170.000000,390.000000,1,15,23043,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.230000e+06,307000.000000,69.732689,17.838533,6599,0 days 00:00:00
23065,300,13,193.000000,297.000000,1,15,23047,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.253000e+06,214000.000000,69.622429,15.425930,6600,0 days 00:00:00
23066,300,18,218.000000,287.000000,1,15,23052,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.278000e+06,204000.000000,69.404030,15.117315,6601,0 days 00:00:00
23067,300,30,284.000000,349.225721,3,15,23064,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.344000e+06,266225.720790,68.748089,16.479746,6602,0 days 00:00:00


In [6]:
# # Changing time cell to timedelta
# time_cell = track['time_cell'].astype('timedelta64[ns]').values
# time_cell

### CALCULATING FOR DISTANCE

In [7]:
distance = []
for i in range(len(track)):
    #lat_andenes = 69.141599 # latitude of Andenes
    lat_andenes = 69.2412 # latiude of the radar
    #lon_andenes = 15.684104 # latitude of Andenes
    lon_andenes = 16.0030 # longitude of where the radar is located
    
# 69.2412° N, 16.0030° E
    
    distances = tobac.analysis.haversine(track['latitude'][i], track['longitude'][i], lat_andenes, lon_andenes)
    distance.append(distances)

In [8]:
distance[:10]
# np.max(distance)

[163.80169363443127,
 125.20961670185432,
 123.29125998307515,
 119.98267085603582,
 115.54011938797738,
 110.07866776406749,
 107.55129320993021,
 122.97944145460625,
 122.40005482235608,
 147.60176073104904]

In [9]:
%%time
# apped distance to track dataframe
track['distance'] = pd.Series(distance)

CPU times: user 3.83 ms, sys: 0 ns, total: 3.83 ms
Wall time: 3.83 ms


In [10]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,1,110.987364,214.505783,38,15,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.170987e+06,131505.783191,70.424947,13.466409,1,0 days 00:00:00,163.801694
1,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00,125.209617
2,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00,123.291260
3,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671
4,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00,115.540119


## _cells that lived for at least 10 mins old_ 

In [11]:
%%time
# cell_one_hour = np.timedelta64( 600, 's') #10 minutes in seconds
cell_times = np.timedelta64( 600, 's') #10 minutes in seconds

#convert time_cell to seconds
time_cell = track['time_cell'].astype('timedelta64[ns]').values

# track cell that live for at least 10 minutes long
track_close_to_AMF1 = track[ time_cell >= cell_times ]
# track_close_to_AMF1 = track[ ((track['distance'] <= 95) & (track['time_cell'].values >= cell_one_hour)) ]
track_close_to_AMF1= track_close_to_AMF1.drop_duplicates(subset=['cell']) # remove duplicates of cell_ids
track_close_to_AMF1.head()

CPU times: user 47.4 ms, sys: 40 µs, total: 47.5 ms
Wall time: 49.3 ms


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
3,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671
11,2,6,144.642616,212.252391,12,15,170,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.204643e+06,129252.390947,70.120521,13.355260,4,0 days 00:10:00,141.597639
22,2,7,151.997879,380.783284,8,15,171,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.211998e+06,297783.284436,69.906418,17.667170,6,0 days 00:10:00,98.295699
44,2,27,262.754401,325.077747,16,15,191,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,242077.747280,68.964541,15.949893,20,0 days 00:10:00,30.868930
51,2,30,275.508244,327.186675,6,15,194,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.335508e+06,244186.674794,68.846875,15.968815,23,0 days 00:10:00,43.916249


In [12]:
# total track cells found.
len(track_close_to_AMF1) 

2511

# tracking all cells that were at least 10 minutes old back in time to get all the cells

In [13]:
%%time
combined_tracks_AMF1 = []
for index, cell_id in enumerate(np.unique(track_close_to_AMF1['cell'].values)): # looped through tracks found with AMF1 site
    if cell_id in track['cell']: # checking if those cells(tracks) are within the original tracked dataframe
        track_i = track[track['cell'] == cell_id] # display it if it is within the tracked dataframe
        combined_tracks_AMF1.append(track_i) # append the results you get a new dataFrame calld combined_tracks_AMF1

# Concatenate the dataframes(combined_tracks_AMD) along the rows axis with keys
# tracks_closer_AMF1 = pd.concat(combined_tracks_AMF1, keys=['df1', 'df2', 'df3', 'df4', 'df5', 'df6'])
tracks_closer_AMF1 = pd.concat(combined_tracks_AMF1)
tracks_closer_AMF1

CPU times: user 1.02 s, sys: 24.5 ms, total: 1.04 s
Wall time: 1.06 s


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
1,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00,125.209617
2,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00,123.291260
3,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671
4,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00,115.540119
5,4,5,148.000000,253.920901,3,15,336,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.208000e+06,170920.901252,70.064303,14.426429,2,0 days 00:20:00,110.078668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23033,299,4,144.743717,239.312025,4,15,23004,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.204744e+06,156312.025471,70.103689,14.055373,6578,0 days 00:05:00,122.013418
23034,300,7,146.461260,240.148842,5,15,23041,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.206461e+06,157148.841602,70.087558,14.073848,6578,0 days 00:10:00,120.179000
23044,298,31,256.000000,174.366873,2,15,22993,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.316000e+06,91366.873305,69.124388,12.259164,6586,0 days 00:00:00,148.655096
23045,299,27,256.815629,176.111734,4,15,23027,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.316816e+06,93111.734429,69.116329,12.301452,6586,0 days 00:05:00,147.098363


In [14]:
# resetting the index
tracks_closer_AMF1_index_reset = tracks_closer_AMF1.reset_index(drop=True)
tracks_closer_AMF1_index_reset.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00,125.209617
1,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00,123.291260
2,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671
3,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00,115.540119
4,4,5,148.000000,253.920901,3,15,336,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.208000e+06,170920.901252,70.064303,14.426429,2,0 days 00:20:00,110.078668


In [15]:
# len(tracks_closer_AMF1_index_reset)

# Algorithm to calculate the corresponding reflectivity values (calculate for the max and mean dBZ) and the area

In [16]:
unique_cell_ids = track_close_to_AMF1['cell'].values # getting the cells_tracked
len(unique_cell_ids)

2511

In [17]:
# len(np.unique(tracks_closer_AMF1_index_reset['cell']))
# Mask_ERF = iris.load('./saved-files/Mask_ERF_iris.nc')[0]
# Mask_ERF

In [18]:
%%time
# all masked cell in iris cube
masked_tracked_cell = []
for i, unique_value in enumerate(unique_cell_ids):
    # get masked cell based on cell id
    get_mask_cell = tobac.utils.mask_cell(mask=Mask_ERF, cell=unique_value, track=track, masked=False) 
    masked_tracked_cell.append(get_mask_cell)

CPU times: user 6.64 s, sys: 55.8 ms, total: 6.7 s
Wall time: 6.93 s


In [19]:
# masked_tracked_cell[:10]

In [21]:
%%time
# Convert the segmented/mask cells from iris cube to xrray
masked_values_to_xarray = []
for i, items in enumerate(masked_tracked_cell):
    mask_convert_xr = xr.DataArray.from_iris(masked_tracked_cell[i]) # convert iris cube to xarray data array
    masked_values_to_xarray.append(mask_convert_xr) # append the results to list
    # print(mask_convert_xr)

CPU times: user 5.27 s, sys: 30.6 ms, total: 5.3 s
Wall time: 5.5 s


In [22]:
%%time
# Getting the original masked reflectivity values
slice_cell_times = []
for i, cell_id in enumerate(np.unique(tracks_closer_AMF1['cell'])):
    cell_ids = tracks_closer_AMF1_index_reset[tracks_closer_AMF1_index_reset['cell'] == cell_id] 
    first_time = cell_ids['timestr'][::].values[0] # get first time (start time)
    last_time = cell_ids['timestr'][::-1].values[0]  # get the last time value (end time)
    # slice values based on time
    slice_cell_time = masked_values_to_xarray[i].sel(time=slice(first_time, last_time))
    slice_cell_times.append(slice_cell_time)
    # print(slice_cell_time)

CPU times: user 3.99 s, sys: 34.3 ms, total: 4.02 s
Wall time: 4.14 s


In [23]:
# slice_cell_times[:5]

In [24]:
# type(slice_cell_times)

In [25]:
# %%time
# # Create an empty Dataset
# dataset = xr.Dataset()

# # Add each DataArray to the Dataset as a separate variable
# for index, data_array in enumerate(slice_cell_times):
#     var_name = f'data_array_{index}'  # Create a variable name for each DataArray
#     dataset[var_name] = data_array  # Add the DataArray to the Dataset

# # Save the Dataset as a NetCDF file
# dataset.to_netcdf('all_data_arrays.nc')

In [26]:
%%time
equivalent_reflectivity_factors = 10**((equivalent_reflectivity_factor/10))
# equivalent_reflectivity_factors = 10*np.log10(Z)

CPU times: user 4.22 s, sys: 277 ms, total: 4.5 s
Wall time: 4.81 s


## algorithm to extract the speed, area, maximum, minimum and mean reflectivity values based on cell_id and specific times

In [27]:
%%time
min_reflectivities = []; mean_reflectivities = []; max_reflectivities = []; areas = [];
for i, cell_id in enumerate(np.unique(tracks_closer_AMF1['cell'])):
    #################################
    import warnings
    warnings.filterwarnings('ignore')
    #################################
    cell_ids = tracks_closer_AMF1_index_reset[tracks_closer_AMF1_index_reset['cell'] == cell_id]  # dataframe based on cell_id
    first_time = cell_ids['timestr'][::].values[0] # get first time (start time)
    last_time = cell_ids['timestr'][::-1].values[0]  # get the last time value (end time)
    
#     # # # # Calculating area
    # area = tobac.analysis.calculate_area(track, Mask_ERF)
#     # area = tobac.analysis.calculate_area(cell_ids, masked_tracked_cell[i], method_area=None)
#     area = np.absolute(area['area']/1000000) # area in kilometers
#     areas.append(area)
    
    # area1 = np.divide(area['area'],1000000)
    # area['area($km^2$)'] = area1


    # reflectivity_masked based on times
    reflectivity_masked = equivalent_reflectivity_factors.sel(time=slice(first_time, last_time))
    
    # LOOPING THROUGH THE AND GETTING THE VALUES
    for index in (range(len(reflectivity_masked))):
        mean_ref = reflectivity_masked[index].mean().item() # get mean values of reflectivity # in dBZ values
        # # Convert from dBZ to Z values
        # mean_ref_Z = 10**(mean_ref / 10)
        # # Convert it back to dBZ units
        # mean_ref_dBZ = 10*np.log10(mean_ref_Z)
        #####################################
        max_ref = reflectivity_masked[index].max().item() # get max values of reflectivity
        
        mean_reflectivities.append(mean_ref)
        max_reflectivities.append(max_ref)
        
# concatenate the area values to one series
# area1 = pd.concat(areas)

# calculate velocity
# velocity = tobac.analysis.calculate_velocity(tracks_closer_AMF1_index_reset)

CPU times: user 31.9 s, sys: 83.4 ms, total: 32 s
Wall time: 33.7 s


In [28]:
# converting the mean Z unit back to dBZ
mean_reflectivities1 = 10*np.log10(mean_reflectivities)

# converting the max Z unit back to dBZ
max_reflectivities1 = 10*np.log10(max_reflectivities)

In [29]:
np.min(mean_reflectivities1), np.max(mean_reflectivities1)

(12.70891348332567, 17.0430868015557)

In [30]:
tracks_closer_AMF1_index_reset

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00,125.209617
1,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00,123.291260
2,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671
3,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00,115.540119
4,4,5,148.000000,253.920901,3,15,336,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.208000e+06,170920.901252,70.064303,14.426429,2,0 days 00:20:00,110.078668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17800,299,4,144.743717,239.312025,4,15,23004,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.204744e+06,156312.025471,70.103689,14.055373,6578,0 days 00:05:00,122.013418
17801,300,7,146.461260,240.148842,5,15,23041,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.206461e+06,157148.841602,70.087558,14.073848,6578,0 days 00:10:00,120.179000
17802,298,31,256.000000,174.366873,2,15,22993,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.316000e+06,91366.873305,69.124388,12.259164,6586,0 days 00:00:00,148.655096
17803,299,27,256.815629,176.111734,4,15,23027,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.316816e+06,93111.734429,69.116329,12.301452,6586,0 days 00:05:00,147.098363


In [31]:
# append max min and mean reflectivity values based on each cell id and times to the tracks_closer_AMF1_index_reset dataframe
# tracks_closer_AMF1_index_reset['speed'] = velocity
# tracks_closer_AMF1_index_reset['area1'] = area1
tracks_closer_AMF1_index_reset['mean dBZ'] = pd.Series(mean_reflectivities1)
tracks_closer_AMF1_index_reset['max dBZ'] = pd.Series(max_reflectivities1)

In [32]:
tracks_closer_AMF1_index_reset

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ
0,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00,125.209617,13.245867,34.567482
1,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00,123.291260,12.974534,35.577388
2,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671,12.780750,34.063904
3,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00,115.540119,12.708913,36.584485
4,4,5,148.000000,253.920901,3,15,336,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.208000e+06,170920.901252,70.064303,14.426429,2,0 days 00:20:00,110.078668,12.845599,38.092506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17800,299,4,144.743717,239.312025,4,15,23004,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.204744e+06,156312.025471,70.103689,14.055373,6578,0 days 00:05:00,122.013418,15.240493,41.116443
17801,300,7,146.461260,240.148842,5,15,23041,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.206461e+06,157148.841602,70.087558,14.073848,6578,0 days 00:10:00,120.179000,14.702386,40.106802
17802,298,31,256.000000,174.366873,2,15,22993,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.316000e+06,91366.873305,69.124388,12.259164,6586,0 days 00:00:00,148.655096,15.301029,39.603226
17803,299,27,256.815629,176.111734,4,15,23027,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.316816e+06,93111.734429,69.116329,12.301452,6586,0 days 00:05:00,147.098363,15.240493,41.116443


In [33]:
tracks_closer_AMF1_index_reset.to_csv('../saved-files/threshold-15/final-tracks-threshold-15-excluding-area.csv', index=False)

# END 